In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc,  font_manager
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import statistics
matplotlib.rcParams['axes.unicode_minus'] = False
fontname = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=fontname).get_name()
rc('font', family=font_name)

In [12]:
get_data = pd.read_csv('../data/TEMP/경락가격/양파/양파 월별 데이터와 평균편차(전국포함).csv', index_col=None)

In [3]:
get_data.head()

,Date,서울 총거래물량,서울 총거래금액,서울 평균단가,서울 평균단가 편차,부산 총거래물량,부산 총거래금액,부산 평균단가,부산 평균단가 편차,대구 총거래물량,...,대전 총거래물량,대전 총거래금액,대전 평균단가,대전 평균단가 편차,광주 총거래물량,광주 총거래금액,광주 평균단가,광주 평균단가 편차,전국 평균단가,전국 평균단가 편차
0,2020-01-01,196129.0,166768400,850.299548,-33.010400,1242260.0,996253350,801.968469,18.116979,2276705.7,...,2946856.0,2380499400,807.809883,84.705534,148872.0,115049300,772.806841,5.149676,802.443257,18.231076
1,2020-02-01,161702.0,172359880,1065.910626,-248.621478,1262447.0,1200651900,951.051331,-130.965884,2582537.8,...,2820738.0,2802490210,993.530845,72.937114,40260.0,31587000,784.575261,-6.618745,983.914010,-163.239677
2,2020-03-01,174813.0,207078050,1184.568939,-367.279792,1199742.8,1052491300,877.264110,-57.178663,2244144.0,...,2691912.0,2660264800,988.243598,59.169045,504657.0,402889550,798.343330,-20.386814,916.013848,-95.339516
3,2020-04-01,10802310.0,9554894100,884.523227,-67.234080,356549.0,263060050,737.794945,82.290502,1166191.4,...,4174226.0,3602379110,863.005288,77.353000,1195291.0,932517480,780.159375,-2.202859,862.959390,-42.285057
4,2020-05-01,12249069.0,8503744300,694.235970,123.053177,803859.0,467878100,582.040010,238.045438,996047.7,...,4766306.0,3248563300,681.568347,215.767781,1391894.0,893240450,641.744594,136.211922,676.211993,144.462340


In [13]:
get_data['전국 총거래물량'] = get_data['서울 총거래물량'] + get_data['부산 총거래물량'] + get_data['대구 총거래물량'] + get_data['대전 총거래물량'] + get_data['광주 총거래물량']
get_data['전국 총거래금액'] = get_data['서울 총거래금액'] + get_data['부산 총거래금액'] + get_data['대구 총거래금액'] + get_data['대전 총거래금액'] + get_data['광주 총거래금액']

In [14]:
seoul_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('서울')])
seoul_data = get_data[seoul_columns]

busan_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('부산')])
busan_data = get_data[busan_columns]

daegu_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대구')])
daegu_data = get_data[daegu_columns]

daejeon_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대전')])
daejeon_data = get_data[daejeon_columns]

gwangju_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('광주')])
gwangju_data = get_data[gwangju_columns]

aways_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('전국')])
aways_data = get_data[aways_columns]

In [6]:
aways_data = aways_data[['Date', '전국 총거래물량', '전국 총거래금액', '전국 평균단가', '전국 평균단가 편차']]

In [7]:
seoul_data = seoul_data.rename(columns={'서울 총거래물량':'총거래물량', '서울 총거래금액':'총거래금액', '서울 평균단가':'평균단가', '서울 평균단가 편차':'평균단가 편차'})
busan_data = busan_data.rename(columns={'부산 총거래물량':'총거래물량', '부산 총거래금액':'총거래금액', '부산 평균단가':'평균단가', '부산 평균단가 편차':'평균단가 편차'})
daegu_data = daegu_data.rename(columns={'대구 총거래물량':'총거래물량', '대구 총거래금액':'총거래금액', '대구 평균단가':'평균단가', '대구 평균단가 편차':'평균단가 편차'})
daejeon_data = daejeon_data.rename(columns={'대전 총거래물량':'총거래물량', '대전 총거래금액':'총거래금액', '대전 평균단가':'평균단가', '대전 평균단가 편차':'평균단가 편차'})
gwangju_data = gwangju_data.rename(columns={'광주 총거래물량':'총거래물량', '광주 총거래금액':'총거래금액', '광주 평균단가':'평균단가', '광주 평균단가 편차':'평균단가 편차'})
aways_data = aways_data.rename(columns={'전국 총거래물량':'총거래물량', '전국 총거래금액':'총거래금액', '전국 평균단가':'평균단가', '전국 평균단가 편차':'평균단가 편차'})

In [8]:
seoul_data['지역'] = '서울'
busan_data['지역'] = '부산'
daegu_data['지역'] = '대구'
daejeon_data['지역'] = '대전'
gwangju_data['지역'] = '광주'
aways_data['지역'] = '전국'

In [9]:
all_data = pd.concat([seoul_data, busan_data, daegu_data, daejeon_data, gwangju_data, aways_data])

In [10]:
all_data.to_csv('../data/TEMP/경락가격/양파/양파 컬럼 통합한 데이터와 표준편차.csv', index=False, encoding='UTF-8')